In [ ]:
import cv2
import pickle
import os
os.environ['GLOB_v'] = '0'
os.environ["GLOG_minloglevel"] = "3"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import gc
import glob
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
#from torch.cuda import empty_cache
import concurrent.futures
from collections import deque

size = 20

E0000 00:00:1742897476.615879  110872 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742897476.620791  110872 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742897476.634169  110872 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742897476.634183  110872 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742897476.634184  110872 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742897476.634185  110872 computation_placer.cc:177] computation placer already registered. Please check linka

# x, y 좌표를 이용한 운동 분류

In [1]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque, Counter
from keras.models import load_model
import threading
import time
import tempfile
import os
from gtts import gTTS

def extract_pose_landmarks(results):
    xyz_list = []

    points = [
        11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28
    ]

    lm = results.pose_landmarks.landmark

    neck_x = (lm[11].x + lm[12].x) / 2
    neck_y = (lm[11].y + lm[12].y) / 2
    #neck_z = (lm[11].z + lm[12].z) / 2

    for idx in points:
        x = lm[idx].x - neck_x
        y = lm[idx].y - neck_y
        #z = lm[idx].z - neck_z

        xyz_list.append([x, y])

    return xyz_list

# ---- TTS Thread ----
class TextToSpeechThread(threading.Thread):
    def __init__(self, text):
        super().__init__()
        self.text = text

    def run(self):
        try:
            tts = gTTS(text=self.text, lang='ko')
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmpfile:
                tts.save(tmpfile.name)
                os.system(f"mpg123 {tmpfile.name}")
                os.remove(tmpfile.name)
        except Exception as e:
            print(f"[TTS Error] {e}")

# ---- Prediction Thread ----
class PredictionThread(threading.Thread):
    def __init__(self, model, sequence, lock):
        super().__init__()
        self.model = model
        self.sequence = sequence
        self.lock = lock
        self.result = None
        self.running = True

    def run(self):
        while self.running:
            with self.lock:
                if len(self.sequence) == 20:
                    input_data = np.array(list(self.sequence)).reshape(1, 20, 24)
                else:
                    input_data = None

            if input_data is not None:
                prediction = self.model.predict(input_data, verbose=0)
                self.result = prediction
            else:
                time.sleep(0.01)

    def stop(self):
        self.running = False

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
mp_pose = mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
sequence = deque(maxlen=20)

execrise_list = ["Standing", "Standing Knee Raise", "Shoulder Press", "Squat", "Side Lunge"]
model = load_model('exercise_classifier.h5')

lock = threading.Lock()
predict_thread = PredictionThread(model, sequence, lock)
predict_thread.start()

last_exercise = None
last_tts_time = 0  # TTS 중복 방지 타이머

try:
    if cap.isOpened():
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            #image = cv2.resize(image, (640, 480))

            results = mp_pose.process(image)
            if results.pose_landmarks is None:
                continue

            landmarks = extract_pose_landmarks(results)

            with lock:
                sequence.append(landmarks)

            if predict_thread.result is not None:
                predict = predict_thread.result
                predict_class = int(np.argmax(predict))
                label = execrise_list[predict_class]

                # 이전에 예측된 운동과 현재 운동이 다를 경우 다시 예측된 운동명 TTS 재생
                if label != last_exercise:
                    if time.time() - last_tts_time > 5:
                        tts_thread = TextToSpeechThread(f"{label}")
                        tts_thread.start()
                        last_exercise = label
                        last_tts_time = time.time()

                # 화면에 예측 표시
                cv2.putText(frame, f"{label}", (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3)

            cv2.imshow('Exercise Classifier', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
finally:
    predict_thread.stop()
    predict_thread.join()
    cap.release()
    mp_pose.close()
    cv2.destroyAllWindows()

2025-04-01 14:01:53.353745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743483713.370481  131408 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743483713.375727  131408 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743483713.470616  131408 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743483713.470637  131408 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743483713.470639  131408 computation_placer.cc:177] computation placer alr

error: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


# x, y좌표 + 관절 각도를 이용한 운동 분류

In [ ]:
import cv2
import numpy as np
import tempfile
import mediapipe as mp
from collections import deque, Counter
import threading
import time
import tempfile
import os
from gtts import gTTS
from tensorflow.keras.models import load_model

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def extract_pose_angles(results, frame):
    lm = results.pose_landmarks.landmark
    image_height, image_width, _ = frame.shape

    neck_x = (lm[11].x + lm[12].x) / 2
    neck_y = (lm[11].y + lm[12].y) / 2
    landmarks = [(lm[i].x - neck_x, lm[i].y - neck_y) for i in range(33)]

    angle_joints = [
        (11, 13, 15), (12, 14, 16),  # 양팔
        (23, 25, 27), (24, 26, 28),  # 양다리
        (13, 11, 23), (14, 12, 24),  # 상체
        (11, 23, 25), (12, 24, 26),  # 골반
    ]

    landmark_points = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]

    xy_features = []
    for i in landmark_points:
        x, y = landmarks[i]
        xy_features.extend([x, y])

    angles = []
    features_per_video = []
    for a, b, c in angle_joints:
        angle = calculate_angle(landmarks[a], landmarks[b], landmarks[c])
        angles.append(angle)

        cx = int(lm[b].x * image_width)
        cy = int(lm[b].y * image_height)
        cv2.putText(frame, f"{int(angle)}", (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    full_feature = xy_features + angles
    features_per_video.append(full_feature)
    
    return features_per_video

class TextToSpeechThread(threading.Thread):
    def __init__(self, text):
        super().__init__()
        self.text = text

    def run(self):
        try:
            tts = gTTS(text=self.text, lang='ko')
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmpfile:
                tts.save(tmpfile.name)
                os.system(f"mpg123 {tmpfile.name}")
                os.remove(tmpfile.name)
        except Exception as e:
            print(f"[TTS Error] {e}")

class PredictionThread(threading.Thread):
    def __init__(self, model, sequence, lock):
        super().__init__()
        self.model = model
        self.sequence = sequence
        self.lock = lock
        self.result = None
        self.running = True

    def run(self):
        while self.running:
            with self.lock:
                if len(self.sequence) == 20:
                    input_data = np.array(list(self.sequence)).reshape(1, 20, 32)
                else:
                    input_data = None

            if input_data is not None:
                prediction = self.model.predict(input_data, verbose=0)
                self.result = prediction
            else:
                time.sleep(0.01)

    def stop(self):
        self.running = False


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
mp_pose = mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
sequence = deque(maxlen=20)

execrise_list = ["Standing", "Standing Knee Raise", "Shoulder Press", "Squat", "Front Lunge"]
model = load_model("exercise_classifier_angle.h5")

lock = threading.Lock()
predict_thread = PredictionThread(model, sequence, lock)
predict_thread.start()

fix_exercise = False
fixed_class = None
last_tts_time = 0
last_exercise = None

try:
    if cap.isOpened():
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # frame = cv2.resize(frame, (1280, 720))

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = mp_pose.process(image)
            if results.pose_landmarks is None:
                continue

            features = extract_pose_angles(results, image)
            with lock:
                sequence.append(features)

            if predict_thread.result is not None:
                predict = predict_thread.result
                predict_class = int(np.argmax(predict))
                label = execrise_list[predict_class]

                # 이전에 예측된 운동과 현재 운동이 다를 경우 다시 예측된 운동명 TTS 재생
                if label != last_exercise:
                    if time.time() - last_tts_time > 5:
                        tts_thread = TextToSpeechThread(f"{label}")
                        tts_thread.start()
                        last_tts_time = time.time()
                        last_exercise = label

                cv2.putText(frame, f"{label}", (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

            cv2.imshow('Exercise Classifier', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
finally:
    predict_thread.stop()
    predict_thread.join()
    cap.release()
    mp_pose.close()
    cv2.destroyAllWindows()


2025-03-27 15:41:18.463071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743057678.479619    4383 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743057678.485218    4383 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743057678.499100    4383 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743057678.499115    4383 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743057678.499117    4383 computation_placer.cc:177] computation placer alr